In [ ]:
import os
import datetime
import json
import time
# web scrape
import requests
import bs4
# aws
import boto3
# data analysis
# pandas

In [ ]:
def CONFIG():
    """"""
    
    cwd = os.getcwd()
    today = datetime.date.today()
    
    return {
        'cwd': cwd,
        'today': today,
        'email': {
            'arn': 'arn:aws:sns:us-east-1:<<AWS ACCOUNT NUMBER>>:craigslist',
            'region': 'us-east-1',
            'subject': 'Craigslist Scraper',
        },
        'search_url': 'https://{CITY}.craigslist.org/d/for-sale/search/sss?hasPic=1&sort=rel&max_price={MAX_PRICE}&min_price={MIN_PRICE}&query={ITEM}&search_distance={DISTANCE}',
    }

In [ ]:
def ITEMS():
    """"""
    
    return [
        {
            'item': 'Fender Jazz Bass',
            'cities': ['washingtondc','richmond'],
            'query': {
                'MIN_PRICE': 300,
                'MAX_PRICE': 900,
                'DISTANCE': 50,
                #'keywords': ['___','__','__'],
            },
        },
    ]

In [ ]:
def HOUSING():
    """"""
    
    return [
        {
            'cities': ['___','___','___'],
            'query': {
                'MAX_PRICE': 1000,
                'MIN_PRICE': 1500,
                'DISTANCE': 50,
                'KEYWORDS': ['___','__','__'],
            },
        },
    ]

In [ ]:
def GET_POSTS(link,keywords=None):
    """"""
    
    links = []
    
    r = requests.get(link)
    soup = bs4.BeautifulSoup(r.text,'html.parser')
    for row in soup.find_all(name='li', attrs={'class':'result-row'}):
        links.append(row.find(name='a',attrs={'class':'result-title hdrlnk'})['href'])
    
    return links

In [ ]:
def COMPILE(posts_dict):
    """"""
    
    s = """"""
    
    for item,posts in posts_dict.items():
        s += 'Posts for {}'.format(item)
        for city,urls in posts.items():
            for url in urls:
                s += '\n - ({CITY}) {URL}'.format(CITY=city,URL=url)
    
    return s

In [ ]:
def SNS(email,msg):
    """"""
    
    sns = boto3.client('sns', region_name=email['region'])
    return sns.publish(
        TopicArn=email['arn'],
        Message=msg,
        Subject=email['subject'],
    )

In [ ]:
if __name__== '__main__':
    
    try:
        # init
        CFG = CONFIG()
        POSTS = {}
        MSG = None
        
        # for each item that i want
        for item in ITEMS():
            name = item['item']
            POSTS[name] = {}
            # for each city listed
            for city in item['cities']:
                # request craigslist search url (using filter parameters)
                params = {'ITEM': name.replace(' ','%20'), 'CITY': city, **item['query']}
                url = CFG['search_url'].format(**params)
                # pull the url for individual posts and store in list
                POSTS[name][city] = GET_POSTS(url)
                
        # compile info from all posts
        MSG = COMPILE(POSTS)
        
        # send out sns notification
        SNS(CFG['email'],MSG)
        
    except Exception as e:
        raise